In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
import os
import csv
import pandas as pd
import wfdb  # Make sure wfdb is installed

output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

# N = normal 
# S = supra-ventricular premature
# V = ventricular escape
# F = fusion of ventricular and normal
# Q = unclassified heartbeats
symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}

for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample

        # Filter out symbols not in symbol_to_category
        filtered_symbols_annotations = [(sym, ann) for sym, ann in zip(symbols, annotations) if sym in symbol_to_category]
        categories = [symbol_to_category[sym] for sym, ann in filtered_symbols_annotations]
        annotations_filtered = [ann for sym, ann in filtered_symbols_annotations]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations_filtered})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("Done")

Done


In [3]:
import os
import pandas as pd
import numpy as np


def process_patient_data(patient_number, data_creation_dir="data_creation"):
    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]
    second_column_name = ecg_df.columns[1] if len(ecg_df.columns) > 1 else None

    sampling_rate = 360  # Hz
    total_window_size_seconds = 1  # Total window size in seconds
    total_window_size_samples = total_window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        # Randomly determine the window split around the annotation point
        before_seconds = np.random.uniform(0, total_window_size_seconds)
        after_seconds = total_window_size_seconds - before_seconds
        before_samples = int(before_seconds * sampling_rate)
        after_samples = int(after_seconds * sampling_rate)
        
        start_point = max(0, annotation_point - before_samples)
        end_point = start_point + total_window_size_samples  # Ensure the window is of the exact expected size
        
        # Adjust the end point if it exceeds the length of the data
        if end_point > len(ecg_df):
            end_point = len(ecg_df)
            start_point = max(0, end_point - total_window_size_samples)  # Adjust start point accordingly
        
        # Process data from the first column
        window_data_first_column = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        
        patient_X.append(window_data_first_column)
        patient_Y.append(category)
        
        # If there's a second column, process it and add as a new entry
        if second_column_name:
            window_data_second_column = ecg_df.iloc[start_point:end_point][second_column_name].to_numpy()
            
            patient_X.append(window_data_second_column)
            patient_Y.append(category)  # Repeat category for the new entry
    
    return patient_X, patient_Y



# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[-0.485, -0.485, -0.425, ..., -0.405, -0.41 , -0.39 ],
       [-0.13 , -0.035,  0.05 , ..., -0.275, -0.28 , -0.27 ],
       [-0.325, -0.305, -0.32 , ..., -0.455, -0.47 , -0.46 ],
       ...,
       [ 0.105,  0.105,  0.1  , ...,  0.015,  0.025,  0.025],
       [-0.15 , -0.155, -0.16 , ..., -0.395, -0.38 ,  0.   ],
       [ 0.   , -0.015, -0.005, ...,  0.075,  0.08 ,  0.   ]])

In [5]:
Y

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [6]:
df_x = pd.DataFrame(X)
df_x

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-0.485,-0.485,-0.425,-0.330,-0.220,-0.070,0.120,0.375,0.620,0.780,...,-0.410,-0.415,-0.410,-0.400,-0.400,-0.395,-0.390,-0.405,-0.410,-0.390
1,-0.130,-0.035,0.050,0.120,0.200,0.310,0.435,0.535,0.580,0.475,...,-0.270,-0.285,-0.285,-0.265,-0.265,-0.255,-0.275,-0.275,-0.280,-0.270
2,-0.325,-0.305,-0.320,-0.320,-0.330,-0.340,-0.335,-0.340,-0.345,-0.355,...,-0.430,-0.450,-0.455,-0.450,-0.435,-0.445,-0.450,-0.455,-0.470,-0.460
3,-0.170,-0.170,-0.170,-0.190,-0.185,-0.180,-0.170,-0.175,-0.190,-0.200,...,-0.390,-0.395,-0.400,-0.390,-0.385,-0.390,-0.400,-0.410,-0.420,-0.405
4,-0.315,-0.315,-0.310,-0.295,-0.290,-0.305,-0.310,-0.320,-0.315,-0.300,...,-0.360,-0.345,-0.355,-0.345,-0.345,-0.345,-0.340,-0.310,-0.305,-0.305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,-0.035,-0.050,-0.050,-0.035,-0.025,-0.010,0.005,0.005,-0.020,-0.030,...,0.060,0.070,0.060,0.045,0.045,0.050,0.070,0.065,0.050,0.055
218984,-0.245,-0.250,-0.250,-0.255,-0.270,-0.285,-0.280,-0.275,-0.280,-0.270,...,-0.335,-0.335,-0.350,-0.350,-0.350,-0.340,-0.335,-0.335,-0.325,-0.335
218985,0.105,0.105,0.100,0.095,0.090,0.070,0.070,0.085,0.090,0.100,...,-0.005,0.010,-0.005,-0.015,-0.030,-0.030,-0.020,0.015,0.025,0.025
218986,-0.150,-0.155,-0.160,-0.155,-0.165,-0.165,-0.180,-0.190,-0.200,-0.205,...,-0.350,-0.350,-0.350,-0.365,-0.390,-0.390,-0.390,-0.395,-0.380,0.000


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_x = scaler.fit_transform(df_x)
df_x = pd.DataFrame(df_x)


In [8]:
import pywt

def madev(d, axis=None):
    'Mean absolute deviation of a signal'
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def wavelet_denoising(x, wavelet='sym4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


df_x = wavelet_denoising(df_x, wavelet='sym4', level=2)
df_x = pd.DataFrame(df_x)

In [9]:
df_y = pd.DataFrame(Y, columns=['ColumnName'])
print(df_y)

       ColumnName
0               N
1               N
2               N
3               N
4               N
...           ...
218983          N
218984          N
218985          N
218986          N
218987          N

[218988 rows x 1 columns]


In [10]:
value_counts_y = df_y['ColumnName'].value_counts()
print(value_counts_y)

ColumnName
N    181262
Q     16086
V     14472
S      5562
F      1606
Name: count, dtype: int64


In [11]:
df_fusionné = pd.concat([df_y, df_x], axis=1)
df_fusionné

,ColumnName,0,1,2,3,4,5,6,7,8,...,350,351,352,353,354,355,356,357,358,359
0,N,0.460138,0.461386,0.460253,0.460607,0.477202,0.499968,0.518139,0.533971,0.556814,...,0.463727,0.465837,0.464782,0.462626,0.460179,0.457802,0.456632,0.456042,0.457622,0.460012
1,N,0.481318,0.489622,0.495315,0.501749,0.515073,0.528525,0.547233,0.563650,0.555354,...,0.473346,0.473882,0.474624,0.475335,0.477251,0.479326,0.478707,0.477832,0.476593,0.476010
2,N,0.460342,0.460821,0.461329,0.461881,0.462464,0.463052,0.463683,0.464299,0.464755,...,0.459428,0.459411,0.459418,0.459445,0.459528,0.459638,0.459697,0.459777,0.459852,0.459977
3,N,0.468364,0.469870,0.471308,0.472799,0.474569,0.476330,0.478223,0.479969,0.480697,...,0.462583,0.462029,0.462137,0.462562,0.463220,0.464043,0.464705,0.465415,0.466189,0.467070
4,N,0.468321,0.468464,0.468601,0.468737,0.468872,0.468998,0.469123,0.469234,0.469294,...,0.467153,0.467290,0.467418,0.467541,0.467666,0.467789,0.467896,0.468000,0.468095,0.468190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,N,0.503083,0.502583,0.502084,0.501554,0.500966,0.500381,0.499752,0.499157,0.498828,...,0.504919,0.504810,0.504679,0.504535,0.504342,0.504130,0.503980,0.503820,0.503670,0.503483
218984,N,0.468149,0.468908,0.469623,0.470406,0.471481,0.472590,0.473844,0.475016,0.475354,...,0.468236,0.467671,0.467310,0.467088,0.467030,0.467084,0.467099,0.467184,0.467330,0.467575
218985,N,0.503410,0.503804,0.504237,0.504711,0.505185,0.505668,0.506179,0.506695,0.507173,...,0.502731,0.502533,0.502519,0.502574,0.502682,0.502821,0.502879,0.502943,0.503006,0.503110
218986,N,0.488453,0.487907,0.486768,0.484926,0.484316,0.484015,0.483311,0.482604,0.482346,...,0.467451,0.468347,0.466394,0.464201,0.460273,0.456953,0.461498,0.468565,0.477426,0.486388


In [12]:
df_fusionné.to_csv('df_fusionné.csv', index=False)